In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import datetime
from libs.sequences import SeismicSequence
from libs.iris import irisRequests
from libs.distributions import Weibull, WeibullMM, GaussianMM, InterTimeDistribution
from importlib import reload  # Python 3.4+
import matplotlib.pyplot as plt

# Model

In [5]:
def get_time_nll_loss(inter_times : torch.Tensor, seq_lengths : torch.Tensor, inter_time_distr : InterTimeDistribution):
    log_prob = inter_time_distr.get_log_prob(inter_times)
    mask = SeismicSequence.pack_sequences_mask(seq_lengths, inter_times.shape[1])
    log_like = (log_prob * mask).sum(-1)  # (N,)
    log_surv = inter_time_distr.get_log_survival(inter_times)  # (N, L)
    end_idx = torch.unsqueeze(seq_lengths,-1)  # (N, 1)
    log_surv_last = torch.gather(log_surv, dim=-1, index=end_idx)  # (N, 1)
    log_like += log_surv_last.squeeze(-1)  # (N,)
    return -log_like

In [4]:
class GRUPointProcess(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_mixtures=1):
        super().__init__()
        self.rnn = nn.GRU(input_dim,hidden_dim,
                          num_layers=1,
                          batch_first=True)
        self.n_mixtures = n_mixtures
        if(self.n_mixtures==1):
            self.weibull_mod = nn.Sequential(
                nn.Linear(hidden_dim, 2),
                nn.Softplus()
            )
        else:
            self.weibull_mod = nn.Sequential(
                nn.Linear(hidden_dim, self.n_mixtures*2),
                nn.Softplus()
            )
            self.mixture_mod = nn.Sequential(
                nn.Linear(hidden_dim, self.n_mixtures),
                nn.Softmax(dim=-1)
            )
    def forward(self,x):
        rnn_output, _ = self.rnn(x)
        # shift forward along the time dimension and pad
        # so we can use it to model the inter times
        context = F.pad(rnn_output[:, :-1, :], (0,0, 1,0))
        weibull_params = self.weibull_mod(context)
        if(self.n_mixtures == 1):
            distr = Weibull(weibull_params[..., 0], weibull_params[..., 1])
        else:
            pip_params = self.mixture_mod(context)
            distr = WeibullMM(pip_params, weibull_params[..., :self.n_mixtures], weibull_params[..., self.n_mixtures:])
        return context, distr

# Data

In [ ]:
# specify regions limits
regions = {}
regions['greece'] = (30, 45,18, 44)
regions['california'] = (30, 41, -125, -113)
regions['japan'] = (20, 50, 120, 150)
regions['italy'] = (35,46,6, 19)

In [ ]:
load_cat = True
region_name = "japan"
region = regions[region_name]
if(load_cat):
    df = pd.read_csv("catalogs/" + region_name + ".csv", parse_dates=['Time'])
else:
    start_time = datetime.datetime(1980, 1, 1, 0, 0, 0)
    end_time =  datetime.datetime(2024, 1, 1, 0, 0, 0)
    download_url =irisRequests.url_events_box(start_time, end_time, region[0], region[1], region[2], region[3], minmag=3, magtype="MW")
    df = pd.read_csv(download_url, sep="|", comment="#")
    df.Time = pd.to_datetime(df.Time, errors='coerce')
    df.dropna(axis=0, inplace=True)
    df.sort_values(by="Time", inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.to_csv("catalogs/japan.csv", index=False)

In [ ]:
train_sequences = SeismicSequence.from_pandas_df(df[df.Time < datetime.datetime(2012, 1, 1, 0, 0, 0, tzinfo=datetime.UTC)], unit='s')

# Training

In [1]:
model = GRUPointProcess(1, 10, n_mixtures=3)

NameError: name 'GRUPointProcess' is not defined

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
epochs = 100
use_random_length = True
duration_days = 3
duration = 60*60*24*duration_days # in seconds
for epoch in range(epochs):
    print(epoch)
    with torch.no_grad():
        seqs = []
        t_start = duration
        max_t_end = train_sequences.t_end
        while(True):
            if(use_random_length):
                t_end = t_start + np.random.exponential()*duration
            else:
                t_end = t_start + duration
            if(t_end > max_t_end):
                break
            sub_seq = train_sequences.get_subsequence(t_start, t_end)
            if(len(sub_seq.arrival_times) > 0):
                seqs.append(sub_seq)
            t_start = t_end
        inter_times, features, lengths = SeismicSequence.pack_sequences(seqs) 
    optimizer.zero_grad()
    input_time_features = inter_times.unsqueeze(-1)
    context, distr = model(input_time_features)
    loss = get_time_nll_loss(inter_times, lengths, distr).mean()
    loss.backward()
    optimizer.step()
    print(loss.item())